In [1]:
# designed to be run after 03-clinical_variables_final. this notebook does some data cleaning/processing. run before -___ notebook.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime
%reload_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [3]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 1.14 s


In [4]:
final_pt_df2['final_bin'].value_counts()

C_neg/A_partial    7867
C_neg/A_full       7401
C_pos/A_full       2438
C_pos/A_partial    1927
Name: final_bin, dtype: int64

time: 6.2 ms


In [5]:
final_pt_df2['final_bin'].value_counts()

C_neg/A_partial    7867
C_neg/A_full       7401
C_pos/A_full       2438
C_pos/A_partial    1927
Name: final_bin, dtype: int64

time: 4.77 ms


# extracting clinical data for our patients
## IMPORTANT, USE THIS TO TUNE TIMEWINDOW OF EXTRACTION AND FOLDER TO SAVE IN

#NOTE ON MY DF NAMING CONVENTION:
origionally when I coded this workbook, it was for 72 hour timewindows, so every dataframe had _72 at the end. this was changed on 6/5/19 and was made more generalizable by finding name of each corresponding df in the df list and using this variable.

In [6]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df, save_boolean


time: 747 µs


### begin pipeline:

# changing my code structure to be a dictionary of dataframes

In [7]:
#folder to save files to:
save_path= str(repository_path)+'/data/cleaned/'
#folder=None


def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_prepped.csv' %(date, df_name)))


time: 6.61 ms


In [8]:
save_path+'%s/'%(folder)

'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/'

time: 1.84 ms


In [9]:
##folder with all clinical variable csv's
allFiles = glob.glob(str(repository_path)+ '/data/raw/%s/'%(folder) + "{}_*.csv".format(date))

time: 1.68 ms


In [10]:
allFiles

['/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_vaso_dose.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_gcs.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_rrt_merged.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_sofa.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_ventcategory.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_cancer_elix.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_uti_all.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr_window/30102019_heightfirstday.csv',
 '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/raw/48_hr

time: 1.7 ms


In [11]:
#making a dictionary of all my dataframes for easier cycling through

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('.csv')[0]) #making an list of all my dataframes in order they appear in file

dfs = {}
i=0
for name in df_list:
    dfs[name] = pd.read_csv(allFiles[i],  index_col=0)
    i+=1

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 15.5 s


In [12]:
df_list

['vaso_dose',
 'gcs',
 'rrt_merged',
 'sofa',
 'ventcategory',
 'cancer_elix',
 'uti_all',
 'heightfirstday',
 'pt_info',
 'sum_elix',
 'weightfirstday',
 'bg_all_nosummary',
 'labs_all_nosummary',
 'vitals_all_nosummary']

time: 2.27 ms


In [13]:
#assigning the appropriate name to each df in a flexible way

indices = [i for i, s in enumerate(df_list) if 'bg' in s]
bg_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'cancer' in s]
cancer_elix_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'uti' in s]
uti_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vent' in s]
vent_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vitals' in s]
vitals_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'vaso' in s]
vaso_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'pt_info' in s]
pt_info_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'gcs' in s]
gcs_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'sum_elix' in s]
sum_elix_df = df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'sofa' in s]
sofa_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'weight' in s]
weight_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'labs' in s]
labs_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'height' in s]
height_df= df_list[indices[0]]

indices = [i for i, s in enumerate(df_list) if 'rrt' in s]
rrt_df= df_list[indices[0]]

time: 66.7 ms


In [14]:
#adding a t_0 to each df that doesn't currently have it

for element in df_list:
    #print(element,':',list(dfs[element]))
    if ('t_0' in list(dfs[element]))==False and 'icustay_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['icustay_id','t_0']], how='left')
    elif ('t_0' in list(dfs[element]))==False and 'hadm_id' in list(dfs[element]) :
        #print("true")
        dfs[element]= pd.merge(dfs[element], final_pt_df2[['hadm_id','t_0']], how='left')
    else:
        print("false")


false
false
false
false
false
false
false
false
false
false
time: 107 ms


In [15]:
for element in df_list:
    print(element,':',list(dfs[element]))

vaso_dose : ['icustay_id', 'charttime', 'endtime', 'vaso_rate', 'vaso_amount', 'amount_uom', 'rate_uom', 'label', 't_0', 'delta']
gcs : ['subject_id', 'hadm_id', 'icustay_id', 'day', 'mingcs', 'gcsmotor', 'gcsverbal', 'gcseyes', 'endotrachflag', 'icu_admit', 't_0', 'approx_charttime', 'admit_plus_day', 'delta', 'uom']
rrt_merged : ['icustay_id', 't_0', 'first_charttime', 'rrt', 'uom']
sofa : ['subject_id', 'hadm_id', 'icustay_id', 'day', 'sofa', 'respiration', 'pao2fio2_vent_min', 'pao2fio2_novent_min', 'coagulation', 'platelet_min', 'liver', 'bilirubin_max', 'cardiovascular', 'rate_dopamine', 'rate_epinephrine', 'rate_norepinephrine', 'rate_dobutamine', 'meanbp_min', 'cns', 'mingcs', 'renal', 'creatinine_max', 'urineoutput', 't_0', 'icu_admit', 'approx_charttime', 'floor_charttime', 'floor_time_var']
ventcategory : ['icustay_id', 'day', 'value', 'uom', 't_0']
cancer_elix : ['subject_id', 'hadm_id', 'icustay_id', 'value', 'label', 'delta', 'uom', 't_0']
uti_all : ['subject_id', 'hadm_i

In [16]:
from parameters import time_var, value_fill, delta_fill, uom_fill
def yn_convert(df, #df in format where each row corresponds to a test, and a patient can have many rows
               label_fill, # value that will be filled to na's
               pt= final_pt_df2,
               time_var=time_var,#'t_0', #
               value_fill=value_fill,#0,
               delta_fill=delta_fill,# pd.to_timedelta('0 days'),
               uom_fill=uom_fill):#'y/n'):

    """
    description: collapses (binarizes) a dataframe where each row corresponds to a test, and a patient can have many rows -> 
    1 row per patient where value is binary variable yes or no a patient has any value within the timewindow (specified in data collection).
    said a different way, for patient this fxn collapses values down to does pt have a non NA value in the clinical time window y/n? 
    
    label_fill: the variable name in the label column of the specified dataframe that will considered for y/n value within timewindow. if any non NA value is present
        it will be considered positive.
    pt: the by patient spreadsheet be be used to supply patient information.
    time_var: the variable used to create the time window of interest.
    value_fill: the variable value that missing values will be filled if the value is not present (default =0) in the origional dataset
    delta_fill: the time delta value that will be filled in if a patient doesn't have any instances of the label_fill.  
    uom_fill: fills in the unit of measurement to this for missing values.
    
    returns a flat 1 row per icustay_id of 1 or 0 if any value was present for the patient.
    """
    
    yn_df = pd.merge(pt[['icustay_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    yn_df.loc[yn_df.loc[:,'value']!=value_fill, 'value']= 1 #squashes all other values into a binary 1 = yes
    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)


time: 20.4 ms


## vaso dose

In [17]:
# #renaming starttime to charttime and dropping endtime

dfs[vaso_df]= dfs[vaso_df].rename(
    columns={'starttime':'charttime','label':'vaso_type'})

time: 24.4 ms


In [18]:
len(dfs[vaso_df])

59474

time: 1.8 ms


In [19]:
dfs[vaso_df].head()

,icustay_id,charttime,endtime,vaso_rate,vaso_amount,amount_uom,rate_uom,vaso_type,t_0,delta
4,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,0.073958,mg,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
8,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,1.049601,mg,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
22,200952.0,2139-09-23 23:12:00,2139-09-24 21:54:00,0.149992,7.990853,mg,mcg/kg/min,norepinephrine,2139-09-23,0 days 23:12:00.000000000
23,200952.0,2139-09-24 21:54:00,2139-09-25 13:17:00,0.201337,7.325205,mg,mcg/kg/min,norepinephrine,2139-09-23,1 days 21:54:00.000000000
34,200952.0,2139-09-23 15:06:00,2139-09-24 00:51:00,2.509001,43.660095,mg,mcg/kg/min,phenylephrine,2139-09-23,0 days 15:06:00.000000000


time: 14.1 ms


In [20]:
#removing units/hour because that is a different use of vasopressin
dfs[vaso_df] = dfs[vaso_df].loc[dfs[vaso_df].loc[:,'rate_uom']!= 'units/hour',:]
dfs[vaso_df] = dfs[vaso_df].loc[dfs[vaso_df].loc[:,'rate_uom']!= 'Uhr',:]

time: 17.2 ms


In [21]:
len(dfs[vaso_df])

59474

time: 2.89 ms


In [22]:
dfs[vaso_df]['rate_uom'].unique()

array(['mcg/kg/min', 'mcgkgmin', 'Umin', 'units/min'], dtype=object)

time: 6.33 ms


#### removing outliers/extreme values

In [23]:
# Use transform to add a column back to the orig df from a groupby aggregation, transform returns a Series with its index aligned to the orig df:
def vaso_outlier_removal(df):
    test_group=(dfs[vaso_df][['vaso_rate','rate_uom','amount_uom','vaso_type']]#.groupby('vaso_type', as_index=False)
         .groupby(['vaso_type','rate_uom'])
             )

    dfs[vaso_df]['std']=test_group.transform(lambda x : x.std())
    dfs[vaso_df]['mean']=test_group.transform(lambda x : x.mean())

    normal_high_value= pd.DataFrame({
        'vaso_type' : ['dobutamine','dopamine','epinephrine','norepinephrine','vasopressin','phenylephrine'],
        'high_value': [40, 20, 0.5, 1, 0.1, 2] #highest values one might expect to see in a clinic, ie above this is likely erroneous
    }) #found from literature, see notes

    dfs[vaso_df] = pd.merge(dfs[vaso_df], normal_high_value, left_on='vaso_type', right_on='vaso_type')

    vaso_dose_72_rmout =(dfs[vaso_df][
        ~((dfs[vaso_df]['vaso_rate'] > dfs[vaso_df]['high_value']) & ((dfs[vaso_df]['vaso_rate']-dfs[vaso_df]['mean'])>= (3*dfs[vaso_df]['std'])))
        ])
    #ie vaso_dose_72_rmout is a dataframe of all rows that excludes rows where vaso rate > literature high value and where vaso_rate >3sd from teh mean
    return(vaso_dose_72_rmout)

time: 23.2 ms


In [24]:
dfs[vaso_df]= vaso_outlier_removal(dfs[vaso_df])
len(dfs[vaso_df]) #52976 ->49340

59135

time: 182 ms


In [25]:
dfs[vaso_df]['vaso_type'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'dopamine',
       'dobutamine', 'vasopressin'], dtype=object)

time: 5.62 ms


In [26]:
#standardizing names, dropping unneeded columns for analysis
dfs[vaso_df]= dfs[vaso_df].drop(['vaso_amount', 'amount_uom','std','mean','high_value'], axis=1)
dfs[vaso_df]= dfs[vaso_df].rename(index=str, columns={'vaso_rate': 'value', 'rate_uom':'uom','vaso_type':'label'})
dfs[vaso_df]['label'].unique()

array(['epinephrine', 'norepinephrine', 'phenylephrine', 'dopamine',
       'dobutamine', 'vasopressin'], dtype=object)

time: 64.8 ms


In [27]:
dfs[vaso_df].head()

,icustay_id,charttime,endtime,value,uom,label,t_0,delta
0,200349.0,2139-06-02 16:31:00,2139-06-02 18:10:00,0.012030,mcg/kg/min,epinephrine,2139-06-02,0 days 16:31:00.000000000
1,200952.0,2139-09-23 15:07:00,2139-09-24 06:43:00,0.020004,mcg/kg/min,epinephrine,2139-09-23,0 days 15:07:00.000000000
2,200977.0,2166-12-02 16:59:00,2166-12-03 05:50:00,0.030015,mcg/kg/min,epinephrine,2166-12-02,0 days 16:59:00.000000000
3,201005.0,2169-10-25 17:25:00,2169-10-25 18:00:00,0.020000,mcgkgmin,epinephrine,2169-10-25,0 days 17:25:00.000000000
4,201005.0,2169-10-25 18:30:00,2169-10-25 19:00:00,0.020000,mcgkgmin,epinephrine,2169-10-25,0 days 18:30:00.000000000


time: 11.4 ms


In [28]:
epinephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='epinephrine']
norepinephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='norepinephrine']
phenylephrine_df= dfs[vaso_df][dfs[vaso_df]['label']=='phenylephrine']
vasopressin_df= dfs[vaso_df][dfs[vaso_df]['label']=='vasopressin']
dopamine_df= dfs[vaso_df][dfs[vaso_df]['label']=='dopamine']
dobutamine_df= dfs[vaso_df][dfs[vaso_df]['label']=='dobutamine']


time: 39.6 ms


In [29]:
#(could be transfered to a different spdsheet for collapsing values)
#y/n convert, seperating out vaso_dose into 6 constitutient dataframes, and for each am collapsing values down to does pt have in time window y/n? 

epinephrine_df=yn_convert(epinephrine_df, label_fill='epinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
norepinephrine_df=yn_convert(norepinephrine_df, label_fill='norepinephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
phenylephrine_df=yn_convert(phenylephrine_df, label_fill='phenylephrine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
vasopressin_df=yn_convert(vasopressin_df, label_fill='vasopressin', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dopamine_df=yn_convert(dopamine_df, label_fill='dopamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
dobutamine_df=yn_convert(dobutamine_df, label_fill='dobutamine', pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)

time: 985 ms


In [30]:
save_df(epinephrine_df, df_name='epinephrine')

/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned/48_hr_window/
time: 409 ms


In [31]:
# pd.DataFrame(epinephrine_df).to_csv(Path(
#     save_path+ '/%s_epinephrine_prepped.csv' %(date)))

save_df(epinephrine_df, df_name='epinephrine')
del(epinephrine_df)

save_df(norepinephrine_df, df_name='norepinephrine')
del(norepinephrine_df)

save_df(phenylephrine_df, df_name='phenylephrine')
del(phenylephrine_df)

save_df(vasopressin_df, df_name='vasopressin')
del(vasopressin_df)

save_df(dopamine_df, df_name='dopamine')
del(dopamine_df)

save_df(dobutamine_df, df_name='dobutamine')
del(dobutamine_df)

del(dfs[vaso_df])


time: 4.04 s


## ELIX

In [32]:
#convert cancer elix to y/n:
dfs[cancer_elix_df]= yn_convert(dfs[cancer_elix_df], label_fill=0, pt= final_pt_df2, value_fill=0, delta_fill=0, uom_fill='y/n', time_var=time_var)
save_df(dfs[cancer_elix_df], df_name='cancer_elix')
del(dfs[cancer_elix_df])

save_df(dfs[sum_elix_df], 'sum_elix')
del(dfs[sum_elix_df])

time: 775 ms


## vitals -

In [33]:
dfs[vitals_df].head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum,t_0,delta
0,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.0,2149-11-10,1 days 00:00:00.000000000
1,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.0,2149-11-10,1 days 19:00:00.000000000
2,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.0,2149-11-10,1 days 19:45:00.000000000
3,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.0,2149-11-10,1 days 20:00:00.000000000
17,9,150750,220597,2149-11-10 10:00:00,mmHg,MeanArtPress,109.0,2149-11-10,0 days 10:00:00.000000000


time: 11.5 ms


In [34]:
dfs[vitals_df] = dfs[vitals_df].rename(index=str, columns={"valueuom":"uom","vitalid":'label', 'valuenum':'value'}) #change valueom to uom
dfs[vitals_df] = dfs[vitals_df].loc[dfs[vitals_df]['label'].notnull(),:]#.count() #removing null values

dfs[vitals_df].loc[dfs[vitals_df].loc[:,'uom']=='BPM','uom']='bpm'

#overall the values are extremely similar and are likely the same thing
#i will combine them.
dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='RespRate') & 
    (dfs[vitals_df]['uom']=='bpm'),'uom']='insp/min'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempC') & 
    (dfs[vitals_df]['uom']=='?C'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF') & 
    (dfs[vitals_df]['uom']=='Deg. F'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF') & 
    (dfs[vitals_df]['uom']=='?F'),'uom']='Deg. C'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempF'),'label']='temperature'

dfs[vitals_df].loc[
    (dfs[vitals_df]['label']=='TempC'),'label']='temperature'

time: 15.5 s


- glucose max
- glucose min
- diasBP min
- heartrate min
- meanart pressure min
- RespRate min
- SYSbp min
- TEMPC min

### most likely erroneous value removal

In [35]:
#erroneous value cutoff summary
## setting a conservative threshold for erroneous values to not skew my data.

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'icustay_id']==228393.0) &
                         (dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']>99999), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']>99998), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='Glucose') &
                         (dfs[vitals_df].loc[:,'value']<15), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='DiasBP') &
                         (dfs[vitals_df].loc[:,'value']<15), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='HeartRate') &
                         (dfs[vitals_df].loc[:,'value'].between(1,29)), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='RespRate') &
                         (dfs[vitals_df].loc[:,'value']<4), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='SysBP') &
                         (dfs[vitals_df].loc[:,'value']<40), 'value'])=np.nan

(dfs[vitals_df].loc[(dfs[vitals_df].loc[:,'label']=='TempC') &
                         (dfs[vitals_df].loc[:,'value']<28), 'value'])=np.nan

time: 2.37 s


In [36]:
dfs[vitals_df] = dfs[vitals_df].loc[dfs[vitals_df]['value'].notnull(),:]#.count() 


time: 3.61 s


In [37]:
save_df(dfs[vitals_df], 'vitals')
del(dfs[vitals_df])


time: 40.4 s


# labs -

In [38]:
dfs[labs_df]= dfs[labs_df].rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 244 ms


In [39]:
dfs[labs_df].groupby('label')['uom'].value_counts() #looks good

label        uom    
ALBUMIN      g/dL       11556
ANION GAP    mEq/L      60366
BANDS        %           4062
BICARBONATE  mEq/L      61523
BILIRUBIN    mg/dL      18798
BUN          mg/dL      61963
CHLORIDE     mEq/L      69231
CREATININE   mg/dL      62214
GLUCOSE      mg/dL      84708
HEMATOCRIT   %          82432
HEMOGLOBIN   g/dL       67039
LACTATE      mmol/L     40121
PLATELET     K/uL       58189
POTASSIUM    mEq/L      88916
PT           SECONDS    41929
PTT          sec        43966
SODIUM       mEq/L      72553
WBC          K/uL       55873
Name: uom, dtype: int64

time: 305 ms


### most likely erroneous value removal

In [40]:
#summary value removal- was explored and coded adhoc, difficult to automate
(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==261887) & 
    (dfs[labs_df].loc[:,'label']=='CHLORIDE')
    & (dfs[labs_df].loc[:,'value']==3.4),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==236290) & 
    (dfs[labs_df].loc[:,'label']=='CHLORIDE')
    & (dfs[labs_df].loc[:,'value']==11.0),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==292769) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==28.1),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==298457) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==48.8),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==234174) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==48.7),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==290264) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==42.0),'value'])=np.nan

(dfs[labs_df].loc[
    (dfs[labs_df].loc[:,'icustay_id']==290264) & 
    (dfs[labs_df].loc[:,'label']=='INR')
    & (dfs[labs_df].loc[:,'value']==22.8),'value'])=np.nan

time: 424 ms


In [41]:
dfs[labs_df]= dfs[labs_df].loc[dfs[labs_df].loc[:,'value'].notnull(),:] #removing null values


time: 81 ms


In [42]:
dfs[labs_df].head()

,subject_id,hadm_id,icustay_id,charttime,label,value,uom,t_0,delta
1,4,185777,294638,2191-03-16 05:42:00,ANION GAP,15.0,mEq/L,2191-03-16,0 days 05:42:00.000000000
2,4,185777,294638,2191-03-17 06:00:00,ANION GAP,15.0,mEq/L,2191-03-16,1 days 06:00:00.000000000
9,4,185777,294638,2191-03-16 05:42:00,BICARBONATE,21.0,mEq/L,2191-03-16,0 days 05:42:00.000000000
10,4,185777,294638,2191-03-17 06:00:00,BICARBONATE,21.0,mEq/L,2191-03-16,1 days 06:00:00.000000000
17,4,185777,294638,2191-03-16 05:42:00,BILIRUBIN,1.9,mg/dL,2191-03-16,0 days 05:42:00.000000000


time: 11.6 ms


In [43]:
#removing unwanted values
unwanted_values= ['HEMATOCRIT','ANION GAP','PT','ALBUMIN']
dfs[labs_df]= dfs[labs_df].loc[~dfs[labs_df].loc[:,'label'].isin(unwanted_values),:]

time: 86.6 ms


# factorizing bands start 
* converting bands into a categorical variable since it is very sparse

In [44]:

# yn_convert_band(max_bands, #df in format where each row corresponds to a test, and a patient can have many rows
def yn_convert_band(df,           
                    label_fill="absent",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n'):
    
    yn_df = pd.merge(pt[['icustay_id','hadm_id','subject_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    criteria0=yn_df.loc[:,'value']==value_fill
    criteria1=pd.to_numeric(yn_df.loc[:,'value'])<=threshold
    criteria2=pd.to_numeric(yn_df.loc[:,'value'])>threshold


    yn_df.loc[criteria1, 'value']= "<{}".format(threshold) 
    yn_df.loc[criteria2, 'value']= ">{}".format(threshold) 
    yn_df.loc[criteria0, 'value']= "absent"

    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)
    

time: 33.3 ms


In [45]:
band_df=dfs[labs_df][dfs[labs_df]['label']=='BANDS']
max_bands=band_df.loc[band_df.groupby('icustay_id', as_index=False)['value'].idxmax(),:]
del(band_df)

time: 595 ms


In [46]:
band_cat=yn_convert_band(df=max_bands,           
                    label_fill="BANDS",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n')

time: 115 ms


In [47]:
#drop bands from lab_df
dfs[labs_df]=dfs[labs_df].drop(dfs[labs_df][dfs[labs_df]['label']=='BANDS'].index)

##dropping charttime, may be problematic later.
dfs[labs_df]=dfs[labs_df].drop('charttime', axis=1)

#appending categorical bands 
dfs[labs_df]=dfs[labs_df].append(band_cat)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


time: 601 ms


In [48]:
dfs[labs_df].loc[dfs[labs_df]['label']=='BANDS','value'].value_counts()

absent    16546
<10        1982
>10        1105
Name: value, dtype: int64

time: 53.2 ms


In [49]:
# pd.DataFrame(dfs[labs_df]).to_csv(Path(
#     save_path+'/%s_labs_prepped.csv' %(date)))

save_df(dfs[labs_df], 'labs')
del(dfs[labs_df])

time: 5.98 s


## vent category -

In [50]:
dfs[vent_df]['icustay_id'].nunique()
#13978 patients with someform of vent data. 

19456

time: 4.73 ms


In [51]:
dfs[vent_df]['uom']='mech/O2/none category'
dfs[vent_df].head()

,icustay_id,day,value,uom,t_0
0,200001.0,0,Oxygen,mech/O2/none category,2181-11-26
1,200001.0,1,Oxygen,mech/O2/none category,2181-11-26
2,200003.0,0,Mech,mech/O2/none category,2199-08-04
3,200003.0,1,Mech,mech/O2/none category,2199-08-04
4,200012.0,0,Oxygen,mech/O2/none category,2153-12-23


time: 11.4 ms


In [52]:
dfs[vent_df]=dfs[vent_df].rename(index=str, columns={'day':'delta'})
dfs[vent_df]['label']='vent_recieved'
dfs[vent_df]['delta']=pd.to_timedelta(dfs[vent_df]['delta'], unit='d')
#dfs[vent_df]= dfs[vent_df].drop(columns=['day'], axis=1) #removing day column

time: 38.2 ms


In [53]:
dfs[vent_df].head()

,icustay_id,delta,value,uom,t_0,label
0,200001.0,0 days,Oxygen,mech/O2/none category,2181-11-26,vent_recieved
1,200001.0,1 days,Oxygen,mech/O2/none category,2181-11-26,vent_recieved
2,200003.0,0 days,Mech,mech/O2/none category,2199-08-04,vent_recieved
3,200003.0,1 days,Mech,mech/O2/none category,2199-08-04,vent_recieved
4,200012.0,0 days,Oxygen,mech/O2/none category,2153-12-23,vent_recieved


time: 11.8 ms


In [54]:
#collapsing into 1 column for 3 days

time: 613 µs


In [55]:
def vent_day_collapser(x):
    if 'Mech' in list(x.unique()):
        x= 'Mech'
    elif 'Oxygen' in list(x.unique()):
        x= 'Oxygen'
    else:
        x='None'
    return(x)

time: 6.17 ms


In [56]:
#collapsing all days into the worst day. 
ventcategory_1day_df= dfs[vent_df].copy()
ventcategory_1day_df['value']=ventcategory_1day_df.groupby('icustay_id',as_index=False)['value'].transform(vent_day_collapser)
ventcategory_1day_df= ventcategory_1day_df.drop_duplicates(['icustay_id','value']).sort_values('icustay_id') #
ventcategory_1day_df= ventcategory_1day_df.loc[ventcategory_1day_df.loc[:,'icustay_id'].isin(icustay_id),:] #had icustay ids not in final cohort, fail safe mesure
#ventcategory_1day_df['label']= 'vent_recieved'
ventcategory_1day_df.head()

,icustay_id,delta,value,uom,t_0,label
0,200001.0,0 days,Oxygen,mech/O2/none category,2181-11-26,vent_recieved
2,200003.0,0 days,Mech,mech/O2/none category,2199-08-04,vent_recieved
4,200012.0,0 days,Oxygen,mech/O2/none category,2153-12-23,vent_recieved
5,200014.0,0 days,Mech,mech/O2/none category,2105-02-17,vent_recieved
7,200019.0,0 days,Mech,mech/O2/none category,2178-07-09,vent_recieved


time: 35.9 s


In [57]:
ventcategory_1day_df['value'].value_counts()

Mech      9958
Oxygen    8525
None       973
Name: value, dtype: int64

time: 9.43 ms


In [58]:
save_df(ventcategory_1day_df, 'ventcategory')
del(ventcategory_1day_df,dfs[vent_df] )

time: 469 ms


## weight and height firstday -
i explored weightdurations and it had more missing values than weightfirstday, so i will use that. we can revisit this if we need longitudinal weights

In [59]:
dfs[weight_df]['uom']='kg'
dfs[weight_df].head()  
#weight column is the conglomerate of weight_admin>weight_daily> weight_echoinhosp> weight_echoprehosp

,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp,uom,t_0
0,200001,61.0,61.000000,NaN,NaN,NaN,kg,2181-11-26
1,200003,77.0,77.000000,77.500000,72.57472,NaN,kg,2199-08-04
2,200012,51.2,51.200001,51.200001,NaN,NaN,kg,2153-12-23
3,200014,62.0,62.000000,NaN,NaN,NaN,kg,2105-02-17
4,200019,65.0,65.000000,NaN,NaN,NaN,kg,2178-07-09


time: 13.7 ms


In [60]:
dfs[weight_df]= dfs[weight_df][dfs[weight_df]['weight'].notnull()]
dfs[weight_df]= dfs[weight_df][['icustay_id','weight','uom']]

dfs[weight_df]['label']= 'weight'
dfs[weight_df]=dfs[weight_df].rename(index=str, columns={'weight':'value'})

#adding the assumed first day delta column to standardize all columns
dfs[weight_df]['delta']=pd.to_timedelta(0,'days')

#adding t_0
dfs[weight_df]= pd.merge(dfs[weight_df], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 128 ms


In [61]:
# pd.DataFrame(dfs[weight_df]).to_csv(Path(
#     save_path+'/%s_weight_prepped.csv' %(date)))
save_df(dfs[weight_df], 'weight')

del(dfs[weight_df])

time: 438 ms


In [62]:
dfs[height_df]['uom']='cm'

time: 3.1 ms


In [63]:
dfs[height_df]= dfs[height_df][dfs[height_df]['height'].notnull()]
dfs[height_df]= dfs[height_df][['icustay_id','height','uom']]

time: 9.15 ms


In [64]:
dfs[height_df]['label']= 'height'
dfs[height_df]=dfs[height_df].rename(index=str, columns={'height':'value'})

#adding the assumed first day delta column to standardize all columns
dfs[height_df]['delta']=pd.to_timedelta(0,'days')
#adding t_0
dfs[height_df]= pd.merge(dfs[height_df], final_pt_df2[['icustay_id',time_var]], left_on='icustay_id', right_on='icustay_id')

time: 105 ms


In [65]:
#heightfirstday
save_df(dfs[height_df], 'height')
del(dfs[height_df])

time: 407 ms


# UTI

In [66]:
dfs[uti_df]['value'].unique()#seems good #all uti within clinical timewindow

array(['NEG', 'MOD', 'POS', 'TR', 'SM', 'LG', ' ', 'N', 'SMALL'],
      dtype=object)

time: 14.4 ms


In [67]:
dfs[uti_df]= dfs[uti_df].loc[(dfs[uti_df].loc[:,'value']!='NEG')&
               dfs[uti_df].loc[:,'value'].notnull(),:] #filter to only pos rows

dfs[uti_df]= dfs[uti_df].loc[(dfs[uti_df].loc[:,'value']!='COMPUTER NETWORK FAILURE. TEST NOT RESULTED.')&
               dfs[uti_df].loc[:,'value'].notnull(),:] #filter to only pos rows
dfs[uti_df]= dfs[uti_df].drop_duplicates(subset=['hadm_id','value','charttime'])

dfs[uti_df].loc[dfs[uti_df].loc[:,'value'].notnull(),'value']= 1
dfs[uti_df].loc[dfs[uti_df].loc[:,'value'].isna(),'value']= 0 

time: 170 ms


In [68]:
dfs[uti_df].head()

,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,label,fluid,category,loinc_code,icd9_code,t_0,delta
132,21,109451.0,51486,2134-09-12 00:08:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,41071,2134-09-12,0 days 00:08:00.000000000
186,21,109451.0,51487,2134-09-12 00:08:00,1,NaN,NaN,Nitrite,Urine,Hematology,5802-4,41071,2134-09-12,0 days 00:08:00.000000000
236,31,128652.0,51486,2108-08-24 16:30:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,3453,2108-08-24,0 days 16:30:00.000000000
790,64,172056.0,51486,2143-03-03 04:33:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,03811,2143-03-03,0 days 04:33:00.000000000
832,62,116009.0,51486,2113-02-15 12:27:00,1,NaN,NaN,Leukocytes,Urine,Hematology,5799-2,1125,2113-02-15,0 days 12:27:00.000000000


time: 23 ms


In [69]:
def uti_categorizer(uti_df):
    "useful to get all rows of days with positive values for patients (if multiple pos in a day there will be only 1 row for that day). ie more longitudinal format "
    
    #gives the max pos or neg value per day for 
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})  
    df_timewindow_perday= pd.merge(df_timewindow_perday,final_pt_df2[['icustay_id','hadm_id', time_var]], left_on='hadm_id', right_on='hadm_id', how='left')
    df_timewindow_perday=df_timewindow_perday.sort_values(['hadm_id','value','delta'], ascending=[True,False,True])
    return(df_timewindow_perday)

def uti_collapser(uti_df, label):
    "collapsing longitudinal data into 1 value. will return pos or neg if patient has a positive uti in their stay. one row per icustay_id"
    
    df_timewindow_perday=uti_df.groupby(['hadm_id','delta'], as_index=False)['value'].agg({'value':'max'})
    first_pos= df_timewindow_perday.drop_duplicates(['hadm_id'])
    collapsed= pd.merge(final_pt_df2[['hadm_id','icustay_id','subject_id', time_var]],first_pos, left_on='hadm_id', right_on='hadm_id', how='left')
    
    collapsed['value']= collapsed['value'].fillna(0)
    collapsed.loc[collapsed.loc[:,'value']==1,'value']= 'pos'
    collapsed.loc[collapsed.loc[:,'value']==0,'value']= 'Neg/Not_tested'
    
    collapsed['delta']= collapsed['delta'].fillna(pd.Timedelta(1, unit='d'))
    
    collapsed['label']= label
    
    collapsed['uom']='pos/neg category'
    return(collapsed)

time: 47.4 ms


In [70]:
uti_nit_pos= dfs[uti_df][dfs[uti_df]['label']=="Nitrite"]
uti_leuk_pos= dfs[uti_df][dfs[uti_df]['label']=="Leukocytes"]

leuk_collapsed= uti_collapser(uti_leuk_pos, 'leukocyte')
nit_collapsed= uti_collapser(uti_nit_pos, 'nitrite')


time: 101 ms


In [71]:
save_df(leuk_collapsed, 'leuk')
del(leuk_collapsed)

save_df(nit_collapsed, 'nit')
del(nit_collapsed)
del(dfs[uti_df])

time: 603 ms


# bloodgas

In [72]:
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df]['value'].notnull(),:]
dfs[bg_df] = dfs[bg_df].rename(index=str, columns={'valueuom':'uom'})
dfs[bg_df]= dfs[bg_df].loc[~(dfs[bg_df].loc[:,'value']=='.'),:]

#may need to remove outliers, haven't done as of 10/22/18
dfs[bg_df].head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta
48,9,150750,220597,2149-11-10 03:04:00,BASEEXCESS,NaN,0,mEq/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
49,9,150750,220597,2149-11-10 03:04:00,CALCIUM,1.14,1.14,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
50,9,150750,220597,2149-11-10 03:04:00,FIO2,60.00,60,NaN,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
51,9,150750,220597,2149-11-10 03:04:00,LACTATE,2.40,2.4,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000
52,9,150750,220597,2149-11-10 03:04:00,PCO2,41.00,41,mm Hg,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000


time: 1.32 s


### most likely erroneous value removal


In [73]:
##calcium
#fixing the calcium errors w/o hard coding
(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==249571) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'valuenum']==94.00),'valuenum'])=0.94#.where('valuenum'==94.00))

(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==249571) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'value']=='094'),'value'])=0.94#.where('valuenum'==94.00))


(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==219600) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'valuenum']==97.00),'valuenum'])=0.97#.where('valuenum'==94.00))

(dfs[bg_df].loc[
    (dfs[bg_df].loc[:,'icustay_id']==219600) & 
    (dfs[bg_df].loc[:,'label']=='CALCIUM')
    & (dfs[bg_df].loc[:,'value']=='097'),'value'])=0.97#.where('valuenum'==94.00))


time: 549 ms


In [74]:
##min chloride
#converting it to a null value without hard coding
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==261887.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==3.4),'value'])=np.nan
#converting it to a null value without hard coding
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==261887.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==3.4),'valuenum'])=np.nan

#changing the values without hard coding. 
(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==236290.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==11.0),'valuenum'])=np.nan

(dfs[bg_df].loc[(dfs[bg_df].loc[:,'icustay_id']==236290.0) &
                         (dfs[bg_df].loc[:,'label']=='CHLORIDE') &
                         (dfs[bg_df].loc[:,'valuenum']==11.0),'value'])=np.nan

time: 354 ms


In [75]:
#peep changes summary: 
dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='PEEP')&
                        (dfs[bg_df]['valuenum']>38),'valuenum']=np.nan #remove this or set to 50?

time: 57.7 ms


In [76]:
#temp changes summary:

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==253821)&
                        (dfs[bg_df]['valuenum']==18.9),
                        'value']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==253821)&
                        (dfs[bg_df]['valuenum']==18.9),
                        'valuenum']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==251788)&
                        (dfs[bg_df]['valuenum']==10.0),
                        'value']= np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='TEMPERATURE')&
                        (dfs[bg_df]['icustay_id']==251788)&
                        (dfs[bg_df]['valuenum']==10.0),
                        'valuenum']= np.nan

time: 453 ms


In [77]:
#fio2 changes summary:

##converting a few values to null, thus removing them from the dataset
dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='FIO2') &
                        (dfs[bg_df].loc[:,'value']=='0'),'value']=np.nan

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label']=='FIO2') &
                        (dfs[bg_df].loc[:,'value']=='-'),'value']=np.nan


##removing all fio2 values between 1-20.9

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(1.0,20.9)),'valuenum']=np.nan


##values between 0-1 were found to be ratios, not %, so converting these to % 

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(1.0,20.9)),'valuenum']=np.nan

fio2_dec= dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(0.0,1.0)),'valuenum']

dfs[bg_df].loc[(dfs[bg_df].loc[:,'label'].isin(['FIO2']))&
                        (dfs[bg_df]['valuenum'].between(0.0,1.0)),'valuenum'] = fio2_dec *100

del(fio2_dec)

time: 582 ms


#### removing null values annotated abov

In [78]:
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df]['value'].notnull(),:]

time: 93.8 ms


### splitting specimen out for vent vs non-vent bg data

In [79]:
#adding specimen tag to filter only arterial samples for vent data. 
specimen_df= dfs[bg_df].loc[dfs[bg_df].loc[:,'label']=='SPECIMEN',['unique_var','label','value']]#unique()
specimen_df=specimen_df.rename(index=str, columns={'value':'specimen'})
specimen_df=specimen_df.loc[specimen_df.loc[:,"specimen"]=='ART',:]

dfs[bg_df]= pd.merge(dfs[bg_df],specimen_df[['unique_var','specimen']], left_on='unique_var', right_on='unique_var', how='left')
bg_ART_nosummary=dfs[bg_df].loc[dfs[bg_df].loc[:,'specimen']=='ART',:]
del(specimen_df)

time: 970 ms


In [80]:
bg_labels=['PH','LACTATE','CALCIUM','TEMPERATURE','POTASSIUM',
           'GLUCOSE','HEMOGLOBIN','SODIUM','CHLORIDE','BICARBONATE']
bg_vent_labels=['PCO2','PaO2','PO2','FIO2','PEEP','O2FLOW']

time: 1.36 ms


In [81]:
#restricting to tests that were chosen to be analysed based on %missingness and clinical relevance
dfs[bg_df]= dfs[bg_df].loc[dfs[bg_df].loc[:,'label'].isin(bg_labels),:]
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label'].isin(bg_vent_labels),:]

time: 144 ms


In [82]:
dfs[bg_df].head()

,subject_id,hadm_id,icustay_id,charttime,label,valuenum,value,uom,unique_var,t_0,delta,specimen
1,9,150750,220597,2149-11-10 03:04:00,CALCIUM,1.14,1.14,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000,ART
3,9,150750,220597,2149-11-10 03:04:00,LACTATE,2.40,2.4,mmol/L,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000,ART
5,9,150750,220597,2149-11-10 03:04:00,PH,7.41,7.41,units,2205972149-11-10 03:04:00,2149-11-10,0 days 03:04:00.000000000,ART
10,9,150750,220597,2149-11-10 21:57:00,CALCIUM,1.05,1.05,mmol/L,2205972149-11-10 21:57:00,2149-11-10,0 days 21:57:00.000000000,ART
13,9,150750,220597,2149-11-10 21:57:00,LACTATE,2.30,2.3,mmol/L,2205972149-11-10 21:57:00,2149-11-10,0 days 21:57:00.000000000,ART


time: 16.6 ms


In [83]:
bg_col=['subject_id','hadm_id','icustay_id','charttime','delta',time_var,'label','valuenum','uom']
bg_ART_nosummary= bg_ART_nosummary[bg_col]
dfs[bg_df]= dfs[bg_df][bg_col]
del(bg_col)

time: 25.6 ms


In [84]:
bg_ART_nosummary['label'].unique()

array(['FIO2', 'PCO2', 'PaO2', 'PEEP', 'O2FLOW'], dtype=object)

time: 8.38 ms


In [85]:
#converting O2 to y/n
o2_flow_df= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='O2FLOW',:]
o2_flow_df=o2_flow_df.rename(index=str, columns={'valuenum':'value'})
o2_flow_df= yn_convert(o2_flow_df, label_fill='o2_flow', time_var=time_var)
o2_flow_df['label']="o2_flow" #fixing label

#removing o2_flow from bg_ART
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='O2FLOW',:]

time: 131 ms


In [86]:
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='PEEP',:]

time: 14.1 ms


The PaO2/FiO2 ratio should be calculated using values that are recorded either at the same time stamp, or when the FIO2 occurs within 2 hours BEFORE the PaO2, not the other way around.
For those with no FiO2 recorded within that time frame, you can assume it’s 0.21 (=21%). The worse ratio will be the lowest PaO2 recorded with FiO2 of 1.0 (=100%). So if that is 12, then it will be 12/1 = 12.

In [87]:
#need to groupby icustay_id and charttime for both. pan
pao2= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='PaO2',:].copy() #n=41799 non-null
fio2= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']=='FIO2',:].copy() #n=12311 non-null
pao2['delta']=pd.to_timedelta(pao2['delta'])
fio2['delta']=pd.to_timedelta(fio2['delta'])
pao2['offset']=pao2['delta'] - pd.to_timedelta(2, unit='h') #pao2 should have the offset, not fio2

time: 233 ms


In [88]:
#removing pao2 and fio2 from bg_ART_nosummary
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='PaO2',:]
bg_ART_nosummary= bg_ART_nosummary.loc[bg_ART_nosummary.loc[:,'label']!='FIO2',:]

time: 23.1 ms


* need to find the fio2 values that occur between(delta-2hr, delta) of pao2

In [89]:
######## testing new shit
pao2_fio2_windowed= pd.merge(pao2,
              fio2[['icustay_id','valuenum','label','charttime','delta']].rename(index=str, columns={'valuenum':'valuenum2','label':'label2','charttime':'charttime2','delta':'delta2'}),
              left_on='icustay_id',
              right_on='icustay_id',
              how='left') #

pao2_fio2_windowed= pao2_fio2_windowed[pd.to_timedelta(pao2_fio2_windowed['delta2']).between(pd.to_timedelta(pao2_fio2_windowed['offset']),pd.to_timedelta(pao2_fio2_windowed['delta']))]

time: 105 ms


In [90]:
#grabes from both
pao2_fio2_windowed['delta']=pd.to_timedelta(pao2_fio2_windowed['delta'])
pao2_fio2_windowed['offset']=pd.to_timedelta(pao2_fio2_windowed['delta'])
ratio_df= pd.merge(pao2, pao2_fio2_windowed[['icustay_id','delta','valuenum2', 'charttime2']], on=['icustay_id','delta'], how='left')
del(pao2_fio2_windowed, pao2,fio2)


time: 94.7 ms


In [91]:
#i merged in all rows that matched the 2hr criteria
## For those with no FiO2 recorded within that time frame, you can assume it’s 0.21 (=21%). 
ratio_df.loc[ratio_df.loc[:,'valuenum2'].isnull(),'valuenum2']= 21
ratio_df['ratio']= ratio_df['valuenum']/(ratio_df['valuenum2']/100)
ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime2']= ratio_df.loc[ratio_df['charttime2'].isnull(),'charttime']
ratio_df['deltadelta']=pd.to_datetime(ratio_df['charttime'])-pd.to_datetime(ratio_df['charttime2'])


time: 63.3 ms


In [92]:
ratio_df['ratio'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])

count    48592.000000
mean       527.487872
std        409.766923
min         14.000000
10%        152.000000
25%        270.000000
50%        423.809524
75%        633.333333
99%       2080.952381
max       3690.476190
Name: ratio, dtype: float64

time: 10.3 ms


In [93]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])

count    48592.000000
mean       137.390229
std         82.942252
min         14.000000
10%         69.000000
25%         85.000000
50%        112.000000
75%        156.000000
99%        448.000000
max        775.000000
Name: valuenum, dtype: float64

time: 9.09 ms


In [94]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])

count    48592.000000
mean       137.390229
std         82.942252
min         14.000000
10%         69.000000
25%         85.000000
50%        112.000000
75%        156.000000
99%        448.000000
max        775.000000
Name: valuenum, dtype: float64

time: 8.18 ms


In [95]:
### all of the duplicate PaO2 with different FIo2. if a pao2 value has 2 fio2 values that match the 2 hour window, then the one closest time to the fio2 value is chosen. 
ratio_df=ratio_df.sort_values(['icustay_id','delta','deltadelta']).drop_duplicates(['icustay_id','delta','ratio'])
#43715 ->42455

time: 38.2 ms


In [96]:
ratio_df['ratio'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])

count    47227.000000
mean       536.776013
std        411.477655
min         14.000000
10%        158.000000
25%        280.952381
50%        433.333333
75%        642.857143
99%       2090.476190
max       3690.476190
Name: ratio, dtype: float64

time: 7.5 ms


In [97]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])


count    47227.000000
mean       137.819701
std         83.306441
min         14.000000
10%         70.000000
25%         85.000000
50%        112.000000
75%        157.000000
99%        449.000000
max        775.000000
Name: valuenum, dtype: float64

time: 7.07 ms


In [98]:
ratio_df['valuenum'].describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.99])

count    47227.000000
mean       137.819701
std         83.306441
min         14.000000
10%         70.000000
25%         85.000000
50%        112.000000
75%        157.000000
99%        449.000000
max        775.000000
Name: valuenum, dtype: float64

time: 6.87 ms


In [99]:
#finding worst value:

time: 392 µs


In [100]:
#QC & finding worst value:
ratio_df[ratio_df['icustay_id']==200014]

,subject_id,hadm_id,icustay_id,charttime,delta,t_0,label,valuenum,uom,offset,valuenum2,charttime2,ratio,deltadelta
11224,9514,127229,200014,2105-02-17 01:08:00,0 days 01:08:00,2105-02-17,PaO2,179.0,mm Hg,-1 days +23:08:00,21.0,2105-02-17 01:08:00,852.380952,00:00:00
11225,9514,127229,200014,2105-02-17 04:35:00,0 days 04:35:00,2105-02-17,PaO2,183.0,mm Hg,0 days 02:35:00,21.0,2105-02-17 04:35:00,871.428571,00:00:00
11226,9514,127229,200014,2105-02-17 11:18:00,0 days 11:18:00,2105-02-17,PaO2,142.0,mm Hg,0 days 09:18:00,21.0,2105-02-17 11:18:00,676.190476,00:00:00
11227,9514,127229,200014,2105-02-17 15:28:00,0 days 15:28:00,2105-02-17,PaO2,155.0,mm Hg,0 days 13:28:00,40.0,2105-02-17 15:28:00,387.500000,00:00:00
11229,9514,127229,200014,2105-02-17 17:11:00,0 days 17:11:00,2105-02-17,PaO2,96.0,mm Hg,0 days 15:11:00,28.0,2105-02-17 17:11:00,342.857143,00:00:00
11228,9514,127229,200014,2105-02-17 17:11:00,0 days 17:11:00,2105-02-17,PaO2,96.0,mm Hg,0 days 15:11:00,40.0,2105-02-17 15:28:00,240.000000,01:43:00
11230,9514,127229,200014,2105-02-17 20:53:00,0 days 20:53:00,2105-02-17,PaO2,95.0,mm Hg,0 days 18:53:00,21.0,2105-02-17 20:53:00,452.380952,00:00:00
11231,9514,127229,200014,2105-02-18 04:01:00,1 days 04:01:00,2105-02-17,PaO2,101.0,mm Hg,1 days 02:01:00,21.0,2105-02-18 04:01:00,480.952381,00:00:00


time: 16 ms


In [101]:
# ratio_df.groupby(['icustay_id'], as_index=False)['ratio'].min() #is this what i want for my pao2/fio2 ratio?

f = {'ratio': 'min','delta':'first'}
ratio_min_df=ratio_df.sort_values(['icustay_id','ratio'],ascending=True).groupby('icustay_id', as_index=False).agg(f)

##it's likely that this variable will not explain any additional variation in y than the ventilation category variable we already have. 
##maybe we can stratify patients with a few categories:

##not on vent
##being on vent with normal pao2/fio2 ratio
## being on vent with low pao2/fio2 ratio

time: 26.9 ms


In [102]:
ratio_min_df.head()

,icustay_id,ratio,delta
0,200003,471.428571,1 days 11:15:00
1,200014,240.000000,0 days 17:11:00
2,200019,447.619048,1 days 05:28:00
3,200030,428.571429,0 days 02:57:00
4,200059,312.000000,0 days 07:41:00


time: 6.26 ms


In [103]:
ratio_min_df.head()

,icustay_id,ratio,delta
0,200003,471.428571,1 days 11:15:00
1,200014,240.000000,0 days 17:11:00
2,200019,447.619048,1 days 05:28:00
3,200030,428.571429,0 days 02:57:00
4,200059,312.000000,0 days 07:41:00


time: 6.49 ms


In [104]:
final_pt_df2.head()

,icustay_id,ssc_charttime,ssc_id,icu_admit,ab_id,ab_start,ab_end,ab_ssc_delta,t_0,t_end,...,subject_id,dod_hosp,dod_ssn,ab_course,org_list,spec_type_list,first_pos_else_neg_ssc,sc_result,final_bin,hadm_id
0,294638,2191-03-16 00:00:00,22,2191-03-16 00:29:31,1213116,2191-03-16,2191-03-22,0 days 00:00:00.000000000,2191-03-16,2191-03-22,...,4,NaN,NaN,full,STAPH AUREUS COAG +,BLOOD CULTURE,2191-03-16 00:00:00,positive,C_pos/A_full,185777
1,220597,2149-11-10 09:40:00,45,2149-11-09 13:07:02,2089126,2149-11-10,2149-11-15,0 days 00:00:00.000000000,2149-11-10,2149-11-15,...,9,2149-11-14,2149-11-14,full,NaN,NaN,2149-11-10 09:40:00,negative,C_neg/A_full,150750
2,232669,2104-08-11 00:00:00,60,2104-08-08 02:08:17,616189,2104-08-11,2104-08-12,0 days 00:00:00.000000000,2104-08-11,2104-08-12,...,12,2104-08-20,2104-08-20,partial,NaN,NaN,2104-08-11 00:00:00,negative,C_neg/A_partial,112213
3,273430,2108-08-05 20:42:00,69,2108-08-05 16:26:09,2572274,2108-08-06,2108-08-08,1 days 00:00:00.000000000,2108-08-06,2108-08-08,...,19,NaN,2109-08-18,partial,NaN,NaN,2108-08-05 20:42:00,negative,C_neg/A_partial,109235
4,217847,2134-09-11 09:35:00,70,2134-09-11 20:50:04,1388217,2134-09-12,2134-09-13,1 days 00:00:00.000000000,2134-09-12,2134-09-13,...,21,2135-02-08,2135-02-08,full,NaN,NaN,2134-09-11 09:35:00,negative,C_neg/A_full,109451


time: 21.3 ms


In [105]:
len(ratio_min_df)

7655

time: 1.78 ms


In [106]:
ratio_min_df= pd.merge(final_pt_df2[['icustay_id','hadm_id','t_0',]],ratio_min_df, left_on='icustay_id',right_on='icustay_id', how='left')
ratio_min_df= ratio_min_df.rename(index=str, columns={'ratio':'value'})

ratio_min_df.loc[ratio_min_df.loc[:,'value'].isnull(),'value']=476

ratio_min_df['uom']= 'PaO2/FIO2 ratio'
ratio_min_df['label']= 'pao2fio2Ratio'

time: 28.5 ms


In [107]:
ratio_min_df.loc[:,'delta']=ratio_min_df.loc[:,'delta'].fillna(pd.to_timedelta('0 days'))

time: 6.91 ms


In [108]:
ratio_min_df['value'].describe()#100 PaO2 on 21% FiO2 = 476 ratio

count    19633.000000
mean       414.078475
std        156.758356
min         14.000000
25%        351.666667
50%        476.000000
75%        476.000000
max       2490.476190
Name: value, dtype: float64

time: 6.71 ms


In [109]:
ratio_min_df.head() #100 PaO2 on 21% FiO2 = 476 ratio

,icustay_id,hadm_id,t_0,value,delta,uom,label
0,294638,185777,2191-03-16,476.000000,0 days 00:00:00,PaO2/FIO2 ratio,pao2fio2Ratio
1,220597,150750,2149-11-10,150.000000,0 days 03:04:00,PaO2/FIO2 ratio,pao2fio2Ratio
2,232669,112213,2104-08-11,357.142857,1 days 00:48:00,PaO2/FIO2 ratio,pao2fio2Ratio
3,273430,109235,2108-08-06,552.380952,1 days 06:11:00,PaO2/FIO2 ratio,pao2fio2Ratio
4,217847,109451,2134-09-12,116.000000,0 days 09:21:00,PaO2/FIO2 ratio,pao2fio2Ratio


time: 19.1 ms


# factorizing pco2 start 

In [110]:

# yn_convert_band(max_bands, #df in format where each row corresponds to a test, and a patient can have many rows
def yn_convert_pco2(df=max_bands,           
                    label_fill="absent",
                    threshold=10,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n'):
    
    yn_df = pd.merge(pt[['icustay_id','hadm_id','subject_id', time_var]],
                      df[['icustay_id','value','label','uom','delta']],
                     left_on= 'icustay_id',
                     right_on= 'icustay_id',
                      how='left') #merging all icustay_id's with time_var, where value,label,uom, and delta are nan's if no value exists for that icustay. 
    #the idea is that if any value exists then it is pos.

    yn_df['value']= yn_df['value'].fillna(value_fill) #converts na to 0 in above na rows.
    criteria0=yn_df.loc[:,'value']==value_fill
    criteria1=pd.to_numeric(yn_df.loc[:,'value'])<=threshold
    criteria2=pd.to_numeric(yn_df.loc[:,'value'])>threshold


    yn_df.loc[criteria1, 'value']= "<{}".format(threshold) 
    yn_df.loc[criteria2, 'value']= ">{}".format(threshold) 
    yn_df.loc[criteria0, 'value']= "absent"

    yn_df['delta']= yn_df['delta'].fillna(delta_fill)
    yn_df['delta']= pd.to_timedelta(yn_df['delta']) #filling in the time delta to time =0 for filled rows
    yn_df['uom']= yn_df['uom'].fillna(uom_fill)
    yn_df.loc[yn_df.loc[:,'uom']!=uom_fill, 'uom']= uom_fill
    yn_df['label']= yn_df['label'].fillna(label_fill)
    
    return(yn_df)
    

time: 46.1 ms


In [111]:
bg_ART_nosummary= bg_ART_nosummary.rename(
    columns={'valuenum':'value'}) #changing valuenum to value

time: 15.1 ms


In [112]:
pco2_df=bg_ART_nosummary[bg_ART_nosummary['label']=='PCO2']
max_bands=pco2_df.loc[pco2_df.groupby('icustay_id', as_index=False)['value'].idxmax(),:]
del(pco2_df)

time: 1.51 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [113]:
pco2_cat=yn_convert_band(df=max_bands,           
                    label_fill="PCO2",
                    threshold=50,
                    pt= final_pt_df2,
                    time_var='t_0',
                    value_fill=9999,
                    delta_fill=pd.to_timedelta('0 days'),
                    uom_fill='y/n')

time: 142 ms


In [114]:
#drop bands from lab_df
bg_ART_nosummary=bg_ART_nosummary.drop(bg_ART_nosummary[bg_ART_nosummary['label']=='PCO2'].index)


time: 23 ms


In [115]:
##dropping charttime, may be problematic later. 06/13/19
bg_ART_nosummary=bg_ART_nosummary.drop('charttime', axis=1)

time: 1.81 ms


In [116]:
bg_ART_nosummary=bg_ART_nosummary.append(pco2_cat, sort=True)

time: 7.37 ms


In [117]:
bg_ART_nosummary.loc[bg_ART_nosummary['label']=='PCO2','value'].value_counts()

absent    11981
<50        5385
>50        2267
Name: value, dtype: int64

time: 8.41 ms


# factorizing bands end 

In [118]:
save_df(ratio_min_df,'pafaRatio') # minimum pao2:fio2 ratio during a stay
del(ratio_min_df)

save_df(dfs[bg_df],'bg_all') #all bloodgas
del(dfs[bg_df])

save_df(bg_ART_nosummary,'bg_ART') #only ARTERIAL bloodgas
del(bg_ART_nosummary)

save_df(o2_flow_df,'o2_flow') #need to investigate this more
del(o2_flow_df)

time: 3.33 s


# RRT

In [119]:
#removing null values
dfs[rrt_df] = dfs[rrt_df].loc[dfs[rrt_df]['rrt'].notnull(),:]
dfs[rrt_df]['uom']='category'
dfs[rrt_df]['delta']=pd.to_timedelta(
    pd.to_datetime(dfs[rrt_df]['first_charttime'])-
    pd.to_datetime(dfs[rrt_df][time_var]),
    'days')
dfs[rrt_df]['label']= 'rrt'
dfs[rrt_df]=dfs[rrt_df].rename(index=str, columns={'rrt':'value'})

time: 43 ms


### converting to yes/no

In [120]:
dfs[rrt_df].head()

,icustay_id,t_0,first_charttime,value,uom,delta,label
0,294638,2191-03-16,NaN,0.0,category,NaT,rrt
1,220597,2149-11-10,NaN,0.0,category,NaT,rrt
2,232669,2104-08-11,NaN,0.0,category,NaT,rrt
3,273430,2108-08-06,NaN,0.0,category,NaT,rrt
4,217847,2134-09-12,2134-09-12 13:30:00,1.0,category,13:30:00,rrt


time: 12.6 ms


In [121]:
dfs[rrt_df]['icustay_id'].nunique() #

19633

time: 4.11 ms


In [122]:
rrt_yn= yn_convert(dfs[rrt_df],label_fill='rrt', time_var=time_var)

time: 48.2 ms


In [123]:
save_df(rrt_yn, 'rrt')
del(rrt_yn, dfs[rrt_df])

time: 464 ms


# GCS_72

In [124]:
dfs[gcs_df]['uom']='GCS_score'

time: 1.25 ms


In [125]:
list(dfs[gcs_df])

['subject_id',
 'hadm_id',
 'icustay_id',
 'day',
 'mingcs',
 'gcsmotor',
 'gcsverbal',
 'gcseyes',
 'endotrachflag',
 'icu_admit',
 't_0',
 'approx_charttime',
 'admit_plus_day',
 'delta',
 'uom']

time: 2.61 ms


In [126]:
dfs[gcs_df]['label']= 'mingcs'
dfs[gcs_df]['uom']='gcs_score'
dfs[gcs_df]=dfs[gcs_df].rename(index=str, columns={'mingcs':'value'})

time: 14.4 ms


In [127]:
dfs[gcs_df]=dfs[gcs_df][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 3.46 ms


In [128]:
save_df(dfs[gcs_df], 'gcs')
del(dfs[gcs_df])

time: 56.6 ms


# SOFA
i'm going to remove all sofa variables except daily score, as we have other markers for those in our above data
i may later use this as qc check.
also added delta

In [129]:
dfs[sofa_df]['uom']='daily_sofa_score'

#adding day delta column
dfs[sofa_df]=dfs[sofa_df].sort_values(['hadm_id','day',time_var]) #good
dfs[sofa_df]['day_rank']=dfs[sofa_df].groupby('icustay_id')['day'].rank()
dfs[sofa_df]['delta']=pd.to_timedelta((dfs[sofa_df]['day_rank']-1), 'days')

time: 81.2 ms


In [130]:
dfs[sofa_df]['label']= 'daily_sofa'
dfs[sofa_df]=dfs[sofa_df].rename(index=str, columns={'sofa':'value'})

time: 145 ms


In [131]:
dfs[sofa_df]= dfs[sofa_df][['subject_id','hadm_id','icustay_id','delta','label','value',time_var,'uom']]

time: 16.2 ms


In [132]:
dfs[sofa_df].head()

,subject_id,hadm_id,icustay_id,delta,label,value,t_0,uom
88189,54610,100003,209281,0 days,daily_sofa,4,2150-04-18,daily_sofa_score
52007,23018,100007,217937,0 days,daily_sofa,1,2145-03-31,daily_sofa_score
52008,23018,100007,217937,1 days,daily_sofa,4,2145-03-31,daily_sofa_score
52009,23018,100007,217937,2 days,daily_sofa,0,2145-03-31,daily_sofa_score
1346,533,100009,253656,0 days,daily_sofa,5,2162-05-17,daily_sofa_score


time: 12.9 ms


In [133]:
save_df(dfs[sofa_df], 'sofa')
del(dfs[sofa_df])

time: 1.12 s


# patient Demographic variables

In [134]:
dfs[pt_info_df]

,icustay_id,subject_id,t_0,label,value,delta,uom
0,280836,268,2198-02-16,yearsold,65.9848802824042,0 days 00:00:00.000000000,yearsold
1,206613,269,2170-11-05,yearsold,40.0998279019951,0 days 00:00:00.000000000,yearsold
2,219649,275,2170-10-08,yearsold,82.164211885568,0 days 00:00:00.000000000,yearsold
3,204407,279,2164-06-15,yearsold,74.2940118069815,0 days 00:00:00.000000000,yearsold
4,257572,281,2101-10-18,yearsold,60.0142318173752,0 days 00:00:00.000000000,yearsold
5,238023,285,2152-09-22,yearsold,45.3551008314954,0 days 00:00:00.000000000,yearsold
6,260225,286,2176-01-01,yearsold,85.0156237166324,0 days 00:00:00.000000000,yearsold
7,278519,290,2179-01-27,yearsold,74.3841495867873,0 days 00:00:00.000000000,yearsold
8,291141,306,2199-09-16,yearsold,60.9386263847694,0 days 00:00:00.000000000,yearsold
9,265862,307,2162-11-28,yearsold,74.7267591958831,0 days 00:00:00.000000000,yearsold


time: 30.4 ms


In [135]:
##adjusting ages over 90 (which were set to 300 to deidentify) to 90
admit_index=dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='yearsold')].index
age_tf=pd.to_numeric(dfs[pt_info_df].loc[admit_index,'value'])>90
dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='yearsold')&(age_tf),'value']=90.0

time: 46.4 ms


In [136]:
dfs[pt_info_df].loc[admit_index,'value'].value_counts()

90.0                776
63.4081637069993      1
66.5478703386823      1
61.6716203703704      1
49.2782088942125      1
44.4676982406774      1
47.3690400410678      1
85.5589544515426      1
82.7865793976728      1
73.6585454217051      1
65.4810862359622      1
61.153896145461       1
52.8789559408827      1
72.2622747927599      1
44.1619576900652      1
55.8491155854691      1
86.4934200953178      1
77.1439961213781      1
50.9048232755343      1
49.2291726557153      1
76.2256356947296      1
77.8772847745076      1
61.3304719306918      1
73.8797097054275      1
65.1581717240855      1
68.1677385479251      1
20.758970105458       1
88.5625411628261      1
88.568174449261       1
77.1119210586356      1
                   ... 
78.9128267675615      1
51.0118879128958      1
51.7122149656501      1
66.2069385821482      1
56.8568453874819      1
76.708561265749       1
58.239348651355       1
49.4923419715061      1
75.525660348062       1
50.5303281618374      1
71.7062315258448

time: 13.2 ms


In [137]:
#dfs[pt_info_df].loc[(dfs[pt_info_df]['label']=='first_admit_age') & (dfs[pt_info_df]['value']>90)]

time: 480 µs


In [138]:
#date= '22102018'
save_df(dfs[pt_info_df], 'pt_info')

del(dfs[pt_info_df] )

time: 261 ms
